In [1]:
import numpy as np
import h5py 
import pandas as pd

try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
    
import multiprocessing
from contextlib import contextmanager

In [2]:
from starnet.utils.data_utils.augment import convolve_spectrum
from starnet.utils.data_utils.restructure_spectrum import rebin, continuum_normalize, ensure_constant_sampling

/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/pysynphot/locations.py:329: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for ftp://ftp.stsci.edu/cdbs/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/home/nrpu88/jupyter_py3/lib/python3.6/site-packages/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for ftp://ftp.stsci.edu/cdbs/mtab/*_tmt.fits
  'no thermal calculations can be performed. ' + str(e))


In [3]:
# In[2]:
import os


# Define parameters needed for continuum fitting
LINE_REGIONS = [[4210, 4240], [4250, 4410], [4333, 4388], [4845, 4886], [5160, 5200], [5874, 5916], [6530, 6590]]
SEGMENTS_STEP = 10.  # divide the spectrum into segments of 10 Angstroms


# In[3]:


home = os.getenv('HOME')
scratch = os.getenv('SCRATCH')
starnet_data_folder = os.path.join(home, 'StarNet/starnet/data/')
#intrigoss_grid_path = os.path.join(home, 'projects/rrg-kyi/group_writable/spectra/grids/intrigoss/train/') 
#phoenix_grid_path = os.path.join(home, 'projects/rrg-kyi/group_writable/spectra/grids/phoenix.astro.physik.uni-goettingen.de/v2.0/HiResFITS/PHOENIX-ACES-AGSS-COND-2011/train/') 
#phoenix_wave_path = home+'/'+'/projects/rrg-kyi/group_writable/spectra/grids/phoenix.astro.physik.uni-goettingen.de/v2.0/HiResFITS/PHOENIX-ACES-AGSS-COND-2011/'
#ambre_grid_path = os.path.join(home, 'projects/rrg-kyi/group_writable/spectra/grids/AMBRE/train/')
obs_wave_filepath = os.path.join(home, 'projects/rrg-kyi/group_writable/spectra/UVES_4835-5395.npy')
wave_grid_obs = np.load(obs_wave_filepath)

#print(wave_grid_obs)
#print(len(wave_grid_obs))
#wave_grid_obs = np.linspace(4835,5395,num=10000)

In [ ]:
file_name = 'ferre_3500_6000_100'
save_path = scratch + '/' + file_name

with open(save_path, 'rb') as ferre1_file:
 
    # Step 3
    ferre_1_re = pickle.load(ferre1_file)
 
    # After config_dictionary is read from file
    print(ferre_1_re.shape)

In [ ]:
def pre_processor_ferre(wav_ferre,flux_ferre):


# Trim the wavelength and flux arrays according to observed wave grid
    extension = 10  # Angstroms
    wave_min_request = wave_grid_obs[0] - extension
    wave_max_request = wave_grid_obs[-1] + extension
    #wave_indices_intrigoss = (wav_intrigoss > wave_min_request) & (wav_intrigoss < wave_max_request)
    wave_indices_ferre = (wav_ferre > wave_min_request) & (wav_ferre < wave_max_request)
    #wave_indices_ambre = (wav_ambre > wave_min_request) & (wav_ambre < wave_max_request)
    #wav_intrigoss = wav_intrigoss[wave_indices_intrigoss]
    wav_ferre = wav_ferre[wave_indices_ferre]
    #wav_ambre = wav_ambre[wave_indices_ambre]
    #flux_intrigoss = flux_intrigoss[wave_indices_intrigoss]
    flux_ferre = flux_ferre[wave_indices_ferre]
    #flux_ambre = flux_ambre[wave_indices_ambre]

# Degrade resolution
    #err_intrigoss = np.zeros(len(flux_intrigoss))
    err_ferre = np.zeros(len(flux_ferre))
    #err_ambre = np.zeros(len(flux_ambre))
    #_, flux_intrigoss, _ = convolve_spectrum(wav_intrigoss, flux_intrigoss, err_intrigoss, to_resolution=47000)
    _, flux_ferre, _ = convolve_spectrum(wav_ferre, flux_ferre, err_ferre, to_resolution=47000)
    #_, flux_ambre, _ = convolve_spectrum(wav_ambre, flux_ambre, err_ambre, to_resolution=47000)


    
# Rebin to UVES wave grid
    #flux_intrigoss = rebin(wave_grid_obs, wav_intrigoss, flux_intrigoss)
    flux_ferre = rebin(wave_grid_obs, wav_ferre, flux_ferre)
    #flux_ambre = rebin(wave_grid_obs, wav_ambre, flux_ambre)

# Continuum normalize the spectra
    #flux_intrigoss, _ = continuum_normalize(flux_intrigoss, LINE_REGIONS, wave_grid_obs, SEGMENTS_STEP)
    flux_ferre, _ = continuum_normalize(flux_ferre, LINE_REGIONS, wave_grid_obs, SEGMENTS_STEP)
    #flux_ambre, _ = continuum_normalize(flux_ambre, LINE_REGIONS, wave_grid_obs, SEGMENTS_STEP)

    #print('DONE')
    
    return flux_ferre

# Mask telluric lines
#flux_intrigoss = mask_tellurics('telluric_lines.txt', flux_intrigoss, wave_grid_obs
#flux_phoenix = mask_tellurics('telluric_lines.txt', flux_phoenix, wave_grid_obs)
#flux_ambre = mask_tellurics('telluric_lines.txt', flux_ambre, wave_grid_obs)


# In[16]:

In [ ]:


def pre_processor_ferre_parallel(wav, fluxes):

    @contextmanager
    def poolcontext(*args, **kwargs):
            pool = multiprocessing.Pool(*args, **kwargs)
            yield pool
            pool.terminate()

    num_spectra = np.shape(fluxes)[0]
    num_cpu = multiprocessing.cpu_count()
    pool_size = num_cpu if num_spectra >= num_cpu else num_spectra
    print('[INFO] Pool size: {}'.format(pool_size))
    
    pool_arg_list = [(wav, fluxes[i])
                         for i in range(num_spectra)]
    with poolcontext(processes=pool_size) as pool:
            results = pool.starmap(pre_processor_ferre, pool_arg_list)
        
    return results

In [ ]:
ferre_teff = []
ferre_logg = []
ferre_feh = []

fluxes_ferre = []

f_batch_size = 16

f_count_tot = 0
f_ferre_fin = [] #final array to hold all fluxes

for i in ferre_1_re:
    feh = i[1]
    teff = i[2]
    logg = i[3]
    
    ferre_feh.append(feh)
    ferre_teff.append(teff)
    ferre_logg.append(logg)
    
    ferre_flux_sub = []
    ferre_wav_sub = []
    
    for j in i[4:]:
        ferre_wav_sub.append(j[0])
        
        ferre_flux_sub.append(j[1])
     
    #print(len(ferre_wav_sub))
    ferre_wav_sub = np.asarray(ferre_wav_sub)
    ferre_flux_sub = np.asarray(ferre_flux_sub)
    
    fluxes_ferre.append(ferre_flux_sub)
    
    f_count = len(fluxes_ferre)
    
    if f_count >= f_batch_size:
        t_f = []
        t_f=pre_processor_ferre_parallel(ferre_wav_sub,fluxes_ferre)
        for i in t_f:
            f_ferre_fin.append(i)
        f_count_tot+=f_count
        f_count = 0
        fluxes_ferre = []
    
print(f_count_tot)
print(len(f_ferre_fin))

In [ ]:
file_name = 'ferre_flux_p1'

save_path = scratch + '/' + file_name

with open(save_path, 'wb') as ferre1_file:
 
  # Step 3
  pickle.dump(f_ferre_fin, ferre1_file)

print("First 432 saved!!")    
    #ferre_flux.append(pre_processor_ferre(ferre_wav_sub,ferre_flux_sub))
        

In [ ]:
file_name = 'ferre_5750_8000_100'
save_path = scratch + '/' + file_name

with open(save_path, 'rb') as ferre2_file:
 
    # Step 3
    ferre_2_re = pickle.load(ferre2_file)
 
    # After config_dictionary is read from file
    print(ferre_2_re.shape)

In [ ]:
f_batch_size = 30

f_count_tot = 0
#f_ferre_fin = [] #final array to hold all fluxes

for i in ferre_2_re:
    feh = i[1]
    teff = i[2]
    logg = i[3]
    
    ferre_feh.append(feh)
    ferre_teff.append(teff)
    ferre_logg.append(logg)
    
    ferre_flux_sub = []
    ferre_wav_sub = []
    
    for j in i[4:]:
        ferre_wav_sub.append(j[0])
        
        ferre_flux_sub.append(j[1])
     
    #print(len(ferre_wav_sub))
    ferre_wav_sub = np.asarray(ferre_wav_sub)
    ferre_flux_sub = np.asarray(ferre_flux_sub)
    
    fluxes_ferre.append(ferre_flux_sub)
    
    f_count = len(fluxes_ferre)
    
    if f_count >= f_batch_size:
        t_f = []
        t_f=pre_processor_ferre_parallel(ferre_wav_sub,fluxes_ferre)
        for i in t_f:
            f_ferre_fin.append(i)
        f_count_tot+=f_count
        f_count = 0
        fluxes_ferre = []
    
print(f_count_tot)
print(f_ferre_fin.shape)

In [ ]:
file_name = 'ferre_flux_p2'

save_path = scratch + '/' + file_name

with open(save_path, 'wb') as ferre2_file:
 
  # Step 3
  pickle.dump(f_ferre_fin, ferre2_file)

print("First 432+360 saved!!")

In [ ]:
X_ferre = np.asarray(f_ferre_fin)
Y_ferre = np.column_stack([ferre_teff, ferre_logg, ferre_feh])

In [ ]:
print(X_ferre.shape)
print(Y_ferre.shape)

In [ ]:


print(X_ferre[0])
print(X_ferre[1])

In [ ]:
teff = ferre_teff
feh = ferre_feh
logg = ferre_logg
a_M = []
#for i in range(len(feh)):
#    a_M.append(0)

In [ ]:
import os
import h5py
import numpy as np

home = os.getenv('HOME')

filename = 'ferre_test_100K.h5'
preprocessed_spectra_path = os.path.join(home, 'projects/rrg-kyi/group_writable/spectra/preprocessed/')
save_path = preprocessed_spectra_path+filename

with h5py.File(save_path, 'w') as f:  
    f.create_dataset('spectra_starnet_norm', data=np.asarray(X_ferre))
    f.create_dataset('teff',data = np.asarray(teff))
    f.create_dataset('logg',data = np.asarray(logg))
    f.create_dataset('M_H',data = np.asarray(feh))
    #f.create_dataset('a_M',data = np.asarray(a_M))
